In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as sk_metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import random_split

from nam.wrapper import NAMClassifier, MultiTaskNAMClassifier

C:\Users\satch\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random_state = 2016

In [3]:
dataset = pd.read_csv('nam/data/data_recidivism_recid.data', delimiter=' ', header=None)
dataset.columns = ["age", "race", "sex", "priors_count", "length_of_stay", "c_charge_degree", "two_year_recid"]

In [4]:
dataset.head()

,age,race,sex,priors_count,length_of_stay,c_charge_degree,two_year_recid
0,69,6,2,0,1,1,0
1,34,1,2,0,10,1,1
2,24,1,2,4,1,1,1
3,44,6,2,0,1,2,0
4,41,3,2,14,6,1,1


In [5]:
binary = ['sex', 'c_charge_degree']
other = ['age', 'race', 'priors_count', 'length_of_stay']

In [6]:
scaler = MinMaxScaler((-1, 1))
dataset[other] = scaler.fit_transform(dataset[other])
dataset[binary] = dataset[binary] - 1

In [7]:
dataset

,age,race,sex,priors_count,length_of_stay,c_charge_degree,two_year_recid
0,0.307692,1.0,1,-1.000000,-0.9975,0,0
1,-0.589744,-1.0,1,-1.000000,-0.9750,0,1
2,-0.846154,-1.0,1,-0.789474,-0.9975,0,1
3,-0.333333,1.0,1,-1.000000,-0.9975,1,0
4,-0.410256,-0.2,1,-0.263158,-0.9850,0,1
...,...,...,...,...,...,...,...
6167,-0.871795,-1.0,1,-1.000000,-0.9950,0,0
6168,-0.871795,-1.0,1,-1.000000,-0.9950,0,0
6169,0.000000,1.0,1,-1.000000,-0.9975,0,0
6170,-0.615385,-1.0,0,-0.842105,-0.9975,1,0


In [8]:
data_train, data_test = train_test_split(dataset, train_size=0.8, test_size=0.2, random_state=random_state)
X_train, y_train = data_train[other + binary], data_train['two_year_recid']
X_test, y_test = data_test[other + binary], data_test['two_year_recid']

## Single Task NAMs Classification

In [18]:
model = NAMClassifier(
            num_epochs=100,
            num_learners=1,
            metric='auroc',
            early_stop_mode='max',
            monitor_loss=False,
            n_jobs=1,
            random_state=random_state
        )
from time import perf_counter
print("training")
start = perf_counter()
model.fit(X_train, y_train)
print(perf_counter()-start)

training


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 23.83it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(0):                                                                                                              
                    Training Loss: 1.688 |
Epoch(0):           Validation Loss: 0.782 | AUROC: 0.491:   0%| |
                    Training Loss: 1.688 |
                    Validation Loss: 0.782 | AUROC: 0.491:   1%| |
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 25.36it/s]
                                                                                                                       
  0%|                                                       

Epoch(9):                                                                                                              
                    Training Loss: 0.678 |
Epoch(9):           Validation Loss: 0.677 | AUROC: 0.597:   9%| |
                    Training Loss: 0.678 |
                    Validation Loss: 0.677 | AUROC: 0.597:  10%| |
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 25.11it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(10):                                                                                                             
                    Training Loss: 0.677 |
Epoch(10):          Validation Loss: 0.676 | AUROC: 0.610:  10%| 
                    Training Loss: 0.677 |
                    Validati

                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(19):                                                                                                             
                    Training Loss: 0.671 |
Epoch(19):          Validation Loss: 0.671 | AUROC: 0.617:  19%|▏
                    Training Loss: 0.671 |
                    Validation Loss: 0.671 | AUROC: 0.617:  20%|▏
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 22.92it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(20):                                                    

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 24.72it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(29):                                                                                                             
                    Training Loss: 0.677 |
Epoch(29):          Validation Loss: 0.670 | AUROC: 0.621:  29%|▎
                    Training Loss: 0.677 |
                    Validation Loss: 0.670 | AUROC: 0.621:  30%|▎
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 24.90it/s]
                                                                                                                       
  0%|                                                         

...

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:00<00:00, 18.83it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(67):                                                                                                             
                    Training Loss: 0.675 |
Epoch(67):          Validation Loss: 0.668 | AUROC: 0.623:  67%|▋
                    Training Loss: 0.675 |
                    Validation Loss: 0.668 | AUROC: 0.623:  68%|▋
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 20.91it/s]
                                                                                                                       
  0%|                                                         

Epoch(76):                                                                                                             
                    Training Loss: 0.669 |
Epoch(76):          Validation Loss: 0.671 | AUROC: 0.623:  76%|▊
                    Training Loss: 0.669 |
                    Validation Loss: 0.671 | AUROC: 0.623:  77%|▊
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 21.17it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(77):                                                                                                             
                    Training Loss: 0.673 |
Epoch(77):          Validation Loss: 0.668 | AUROC: 0.624:  77%|▊
                    Training Loss: 0.673 |
                    Validation

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 20.75it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(86):                                                                                                             
                    Training Loss: 0.675 |
Epoch(86):          Validation Loss: 0.669 | AUROC: 0.622:  86%|▊
                    Training Loss: 0.675 |
                    Validation Loss: 0.669 | AUROC: 0.622:  87%|▊
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 24.70it/s]
                                                                                                                       
  0%|                                                         

Epoch(95):                                                                                                             
                    Training Loss: 0.665 |
Epoch(95):          Validation Loss: 0.667 | AUROC: 0.632:  95%|▉
                    Training Loss: 0.665 |
                    Validation Loss: 0.667 | AUROC: 0.632:  96%|▉
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 20.78it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(96):                                                                                                             
                    Training Loss: 0.668 |
Epoch(96):          Validation Loss: 0.667 | AUROC: 0.631:  96%|▉
                    Training Loss: 0.668 |
                    Validation

24.113917600014247


In [19]:
pred = model.predict_proba(X_test)
sk_metrics.roc_auc_score(y_test, pred)

0.6492038972750023

## Multitask NAMs Classification

In [20]:
def make_gender_mtl_data(X, y):
    y_male = y.copy()
    y_male[X['sex'] == 1] = np.nan
    y_female = y.copy()
    y_female[X['sex'] == 0] = np.nan
    return pd.concat([y_female, y_male], axis=1)

In [21]:
y_train_mtl = make_gender_mtl_data(X_train, y_train)
y_test_mtl = make_gender_mtl_data(X_test, y_test)

In [22]:
X_train_mtl = X_train.drop(columns=['sex'])
X_test_mtl = X_test.drop(columns=['sex'])

In [23]:
# NaN indicates label missing
y_train_mtl

,two_year_recid,two_year_recid
4819,0.0,NaN
1581,NaN,1.0
0,0.0,NaN
1575,0.0,NaN
1159,1.0,NaN
...,...,...
4604,0.0,NaN
653,0.0,NaN
4691,0.0,NaN
5386,NaN,1.0


In [24]:
model = MultiTaskNAMClassifier(
            num_learners=1,
            patience=60,
            num_epochs=100,
            num_subnets=1,
            metric='auroc',
            monitor_loss=False,
            early_stop_mode='max',
            n_jobs=1,
            random_state=random_state
        )

start = perf_counter()
print("training")
model.fit(X_train_mtl, y_train_mtl)
print(perf_counter()-start)

training


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 25.92it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(0):                                                                                                              
                    Training Loss: 0.841 |
Epoch(0):           Validation Loss: 0.723 | AUROC: 0.496:   0%| |
                    Training Loss: 0.841 |
                    Validation Loss: 0.723 | AUROC: 0.496:   1%| |
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 24.82it/s]
                                                                                                                       
  0%|                                                       

Epoch(9):                                                                                                              
                    Training Loss: 0.678 |
Epoch(9):           Validation Loss: 0.674 | AUROC: 0.567:   9%| |
                    Training Loss: 0.678 |
                    Validation Loss: 0.674 | AUROC: 0.567:  10%| |
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 26.79it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(10):                                                                                                             
                    Training Loss: 0.666 |
Epoch(10):          Validation Loss: 0.673 | AUROC: 0.579:  10%| 
                    Training Loss: 0.666 |
                    Validati

                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(19):                                                                                                             
                    Training Loss: 0.661 |
Epoch(19):          Validation Loss: 0.652 | AUROC: 0.656:  19%|▏
                    Training Loss: 0.661 |
                    Validation Loss: 0.652 | AUROC: 0.656:  20%|▏
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 25.84it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(20):                                                    

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 24.35it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(29):                                                                                                             
                    Training Loss: 0.640 |
Epoch(29):          Validation Loss: 0.635 | AUROC: 0.674:  29%|▎
                    Training Loss: 0.640 |
                    Validation Loss: 0.635 | AUROC: 0.674:  30%|▎
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 24.19it/s]
                                                                                                                       
  0%|                                                         

...

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(67):                                                                                                             
                    Training Loss: 0.648 |
Epoch(67):          Validation Loss: 0.635 | AUROC: 0.670:  67%|▋
                    Training Loss: 0.648 |
                    Validation Loss: 0.635 | AUROC: 0.670:  68%|▋
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00,  9.63it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(68):                                                                                                             
                    Training Loss: 0.640 |
Epoch(68):         

Epoch(76):                                                                                                             
                    Training Loss: 0.650 |
Epoch(76):          Validation Loss: 0.634 | AUROC: 0.679:  76%|▊
                    Training Loss: 0.650 |
                    Validation Loss: 0.634 | AUROC: 0.679:  77%|▊
 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:00<00:00, 18.27it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(77):                                                                                                             
                    Training Loss: 0.639 |
Epoch(77):          Validation Loss: 0.633 | AUROC: 0.673:  77%|▊
                    Training Loss: 0.639 |
                    Validation

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 22.45it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(86):                                                                                                             
                    Training Loss: 0.638 |
Epoch(86):          Validation Loss: 0.631 | AUROC: 0.684:  86%|▊
                    Training Loss: 0.638 |
                    Validation Loss: 0.631 | AUROC: 0.684:  87%|▊
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 22.65it/s]
                                                                                                                       
  0%|                                                         

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(95):                                                                                                             
                    Training Loss: 0.640 |
Epoch(95):          Validation Loss: 0.625 | AUROC: 0.673:  95%|▉
                    Training Loss: 0.640 |
                    Validation Loss: 0.625 | AUROC: 0.673:  96%|▉
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00, 21.95it/s]
                                                                                                                       
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
Epoch(96):                                                                                                             
                    Training Loss: 0.648 |
Epoch(96):         

24.090321699972264


In [25]:
pred = model.predict_proba(X_test_mtl)

In [26]:
# Flatten and remove nans
y_test_mtl_flat = y_test_mtl.to_numpy().reshape(-1)
pred_flat = pred.reshape(-1)

non_nan_indices = y_test_mtl_flat == y_test_mtl_flat 
y_test_mtl_flat = y_test_mtl_flat[non_nan_indices]
pred_flat = pred_flat[non_nan_indices]

In [27]:
sk_metrics.roc_auc_score(y_test_mtl_flat, pred_flat)

0.6968139883572644